## Загрузка и предобработка файла

In [1]:
import pandas as pd

https://drive.google.com/file/d/1zliuQjWn6KRv3-vMkmXgULKd9OXlfAYz/view?usp=sharing

In [2]:
! gdown --id 1zliuQjWn6KRv3-vMkmXgULKd9OXlfAYz


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1zliuQjWn6KRv3-vMkmXgULKd9OXlfAYz
To: /content/perechetka1.csv
100% 7.61k/7.61k [00:00<00:00, 7.35MB/s]


In [3]:
df = pd.read_csv('/content/perechetka1.csv')

In [4]:
df.head()

,number,type,tree,bush,diameter,height,description,action
0,2,Липа,1,NaN,34-36,14,"сух.ветви, с порослью",Вырубить
1,3,поросль,NaN,18,NaN,"1,5-3","клен остр., многоств.",Вырубить
2,6,сирень куст,NaN,1,NaN,4-5,"старовозраст., многоств., с порос.",Вырубить
3,7,поросль,NaN,2,NaN,"1-2,5",клен ясен.,Вырубить
4,7,снежноягодник куст,NaN,1,NaN,1,удовлетв.,Вырубить


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   number       83 non-null     object
 1   type         83 non-null     object
 2   tree         53 non-null     object
 3   bush         31 non-null     object
 4   diameter     53 non-null     object
 5   height       83 non-null     object
 6   description  83 non-null     object
 7   action       83 non-null     object
dtypes: object(8)
memory usage: 5.3+ KB


В столбцах tree и bush есть пропуски - это специфика данных, в tree отмечено количество только деревьев, bush соответственно кустов

In [6]:
df.tree.value_counts()

1               44
2                3
12               1
21               1
4                1
3                1
Кол-во в шт.     1
Дере- вьев       1
Name: tree, dtype: int64

In [7]:
df.loc[df['tree'].isin(['Кол-во в шт.', 'Дере- вьев'])]

,number,type,tree,bush,diameter,height,description,action
41,№№,Наименование,Кол-во в шт.,NaN,"Диаметр,",Высота,Характеристика состояния,Заклю-
42,п/п,пород,Дере- вьев,Кустар ников,см,м,зеленых насаждений,чение


В данные закрались пара лишних строчек, удалим их, и разберемся с типами данных

In [8]:
df = df.loc[~df['tree'].isin(['Кол-во в шт.', 'Дере- вьев'])]

In [9]:
df['tree'] = pd.to_numeric(df['tree'], errors='coerce')
df['bush'] = pd.to_numeric(df['bush'], errors='coerce')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
display('Общее количество деревьев и кустарников')
df.groupby('action', as_index=False)[['tree', 'bush']].sum()

'Общее количество деревьев и кустарников'

,action,tree,bush
0,Вырубить,90.0,316.0


##Получение данных

Тут только вырубка (еще бывает пересадка и сохранение). Значит для расчетов нам нужны следующие данные: 
1. количество поросли и кустарников, 
2. количество деревьев твердолиственных и мягколиственных, 
3. количество деревьев по группам (диаметр от 1 до 10, от 11 до 20 и тд), 
4. является ли дерево многоствольным, число стволов, 
5. диаметр пня. Если указно несколько диаметров, берем бОльший. Диаметр пня у многоствольных больше. 
6. Количество вывозимых порубочных остатков

###Кустарники

In [11]:
def chop_bush_counter(df):
  try:
    df_bush = df[(~df.bush.isna()) & (df.action == 'Вырубить')]\
          [['type', 'bush', 'height', 'description']]
    df_bush['bush_cat'] = df['type'].apply(lambda x: '1поросль' if x == 'поросль' else '2куст')
    df_bush_to_chop = df_bush.groupby('bush_cat', as_index=False)[['bush']].sum()
    return df_bush_to_chop
  except:
    print('Ошибочка')

In [12]:
df_bush_to_chop = chop_bush_counter(df)

In [13]:
df_bush_to_chop

,bush_cat,bush
0,1поросль,288.0
1,2куст,28.0


In [14]:
bush_weigth = df_bush_to_chop.bush[0]*0.05 + df_bush_to_chop.bush[1]*0.1

In [15]:
print(f'Общее количество кустов {df_bush_to_chop.bush.sum()}, вес порубочных остатков {bush_weigth} т.')

Общее количество кустов 316.0, вес порубочных остатков 17.2 т.


###Деревья

Сначала выделим некоторые вспомогательные функции: деление на группы, подсчет ствольности, деление на категории по твердости древесины, и впомогательные данные по весу в тоннах относительно диаметра ствола

In [16]:
#ствольность обычно указана в комментарии
def is_mnogostv(row):
  for i in range(6):
    if str(i) in row:
        return int(i)
  return 1   

In [17]:
def tree_type(row):
  tverd_types = ['Клен', 'Бук', 'Орех', 'Ясень' , 'Яблоня', 'Вишня', 
           'Сухостой', 'Самосев', 'Рябина', 'Акация']
  for tree_type in tverd_types:
    if tree_type.lower() in row.lower():
      return 'Твердолиственное'
  return 'Мягколиственное/хвойное'

In [18]:
def diameter_bins(row):
  return f'до {row // 10 + 1}0'

In [19]:
df_mass = {'до 10' : 0.0794285714285714, 
           'до 20' : 0.277714285714286, 
           'до 30' : 0.763809523809524, 
           'до 40' : 1.53942857142857, 
           'до 50' : 2.67476190476191, 
           'до 60' : 4.01542857142857, 
           'до 70' : 6.57371428571429, 
           'до 80' : 8.21714285714286, 
           'до 90' : 10.6822857142857, 
           'поросль' : 0.05, 
           'куст' : 0.01}

А теперь собственно функция, которая посчитает нам итоговую таблицу c количеством деревьев по категориям и диаметром и количеством порубочных остатков (а также диаметры пней для корчевки).

In [20]:
def chop_tree_counter(df):
  try:
    df_tree = df[(df.bush.isna()) & (df.action == 'Вырубить')]\
              [['type', 'tree', 'diameter', 'height', 'description']]
    df_tree['stvol'] = df.description.apply(is_mnogostv)
    df_tree['diameter'] = df_tree['diameter'].apply(lambda x: int(x.split('-')[-1]))
    df_tree['stvol_count'] = df_tree['stvol'] * df_tree['tree']
    df_tree['stump_diameter'] = df_tree['diameter'] * df_tree['stvol'] + 5
    df_tree['tree_type'] = df_tree['type'].apply(tree_type)
    df_tree['diameter_bins'] = df_tree['diameter'].apply(diameter_bins)
    df_tree['stump_diameter_bins'] = df_tree['stump_diameter'].apply(diameter_bins)
    df_tree['weight'] = df_tree['diameter_bins'].map(df_mass) * df_tree['stvol_count']
    df_tree_grouped = df_tree.groupby(['tree_type', 'diameter_bins'])\
              [['tree', 'stvol_count', 'weight']].sum()
    df_tree_trees = (pd.concat([df_tree_grouped,
           df_tree_grouped.groupby(level=0).sum()
           .assign(claim_type= "total")
           .set_index('claim_type', append=True)]).sort_index()
           .rename(columns={'tree':'Кол-во деревьев', 
                            'stvol_count': 'Кол-во стволов',
                            'weight': 'Тоннаж'}))
    df_stumps = (df_tree.groupby('stump_diameter_bins')[['tree']].sum()
                .rename(columns={'tree':'Кол-во пней'}))
    return df_tree_trees, df_stumps
  except:
    print('Ошибочка')

In [21]:
trees, stumps = chop_tree_counter(df)

In [22]:
trees

Кол-во деревьев  Кол-во стволов  \
tree_type               diameter_bins                                    
Мягколиственное/хвойное total                     10.0            11.0   
                        до 20                      1.0             1.0   
                        до 40                      5.0             6.0   
                        до 50                      3.0             3.0   
                        до 60                      1.0             1.0   
Твердолиственное        total                     80.0            86.0   
                        до 10                     46.0            46.0   
                        до 20                     20.0            22.0   
                        до 30                     11.0            14.0   
                        до 40                      2.0             2.0   
                        до 60                      1.0             2.0   

                                          Тоннаж  
tree_type               diameter_bins             
Мягколиственное/хвойное total          21.554000  
                        до 20           0.277714  
                        до 40           9.236571  
                        до 50           8.024286  
                        до 60           4.015429  
Твердолиственное        total          31.566476  
                        до 10           3.653714  
                        до 20           6.109714  
                        до 30          10.693333  
                        до 40           3.078857  
                        до 60           8.030857

In [23]:
tree_weigh = trees.query('diameter_bins == "total"')['Тоннаж'].sum()

In [24]:
print(f'''Итого вывозимый тоннаж порубочных остатков 
      {tree_weigh.round(2)} от деревьев, 
      {bush_weigth} от кустов, 
      всего {round(tree_weigh + bush_weigth, 2)} тонны''')

Итого вывозимый тоннаж порубочных остатков 
      53.12 от деревьев, 
      17.2 от кустов, 
      всего 70.32 тонны


In [25]:
stumps

,Кол-во пней
stump_diameter_bins,
до 10,5.0
до 110,1.0
до 20,50.0
до 30,20.0
до 40,7.0
до 50,3.0
до 60,2.0
до 80,1.0
до 90,1.0
